# Battery Module Structural Analysis (Python)
This notebook analyzes the SES/UTS load cases for battery segments using Python. 

### 1. Constants and Geometry
We establish the physical foundations and material properties here.

In [42]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [43]:
import math
import pandas as pd

# Conversion Constants
PSI_TO_PA = 6894.76
MM_TO_M = 1/1000
INCH_TO_M = 25.4/1000
G = 9.80665

# 1.1 Module Dimensions
ene_mod_width = 20.3 * MM_TO_M
ene_seg_height = 105.6 * MM_TO_M
ene_mod_depth = 69.5 * MM_TO_M
module_weight = 0.278  # kg

# 1.2 Segment Assembly
ene_segment_depth = 6 * ene_mod_depth 
ene_segment_width = ene_mod_width * 4 
garolite_thickness = (1/16) * INCH_TO_M
segment_modules_weight = module_weight * 23 

# 1.3 Material Properties
garolite_strength = 38000 * PSI_TO_PA
garolite_modulus = 2400000 * PSI_TO_PA
poly_thickness = 3/8 * INCH_TO_M
poly_strength = 93e6  # Pa 
poly_modulus = 2.21e9 # Pa

# 1.4 Area Calculations
top_cross_sec_area = ene_segment_depth * garolite_thickness * 2
depth_cross_sec_area = (ene_segment_width * garolite_thickness * 2) + (ene_seg_height * garolite_thickness * 2)

## 2. Load Cases & Bending Stress
Calculated for 20g vertical and 40g lateral loads.

Assuming a uniformly distributed load where total force $F$ is related to the distributed load $w$ by $F = w \cdot L$ (or $w = F/L$):

**Maximum Bending Moment:**
$$ M_{max} = \frac{w L^2}{8} = \frac{F \cdot L}{8} $$

**Bending Stress:**
$$ \sigma = \frac{M \cdot c}{I} $$

**Maximum Deflection ($\delta_{max}$):**
$$ \delta_{max} = \frac{5 w L^4}{384 E I} = \frac{5 F \cdot L^3}{384 E I} $$

In [ ]:
# 2.1 Vertical Force (20g)
top_force = segment_modules_weight * 20 * G

# 2.2 Bending Moment (Simply Supported Case)
m_a = (top_force * ene_segment_depth) / 8

# 2.3 Moment of Inertia for Lid
i_lid = (ene_segment_width * (poly_thickness**3)) / 12

# 2.4 Stress and SF
stress_bending_lid = (m_a * (poly_thickness / 2)) / i_lid
sf_lid = poly_strength / stress_bending_lid

# 2.5 Deflection Check
delta_max = (5 * top_force * ene_segment_depth**3) / (384 * poly_modulus * i_lid)

print(f"Lid Safety Factor: {sf_lid:.4f}")
#print(f"Max Lid Deflection: {delta_max * 1000:.2f} mm")

Lid Safety Factor: 1.7468


## 3. Fasteners: Tension, Shear, and Bearing
Checking both bolt strength and the Garolite's resistance to the bolt shank (Bearing Stress).

In [45]:
# 3.1 Bolt Geometry (M6)
dia = 6.0 * MM_TO_M
pitch = 1.0 * MM_TO_M
dia_pitch = dia - 0.649519 * pitch
dia_root = dia - 1.226869 * pitch
area_bolt_tensile = (math.pi / 4) * ((dia_pitch + dia_root) / 2)**2
max_tensile_fastener = 827370875 * area_bolt_tensile # Pa

# 3.2 SF Calculations (Assuming 2 bolts per end)
sf_bolt_tension = (max_tensile_fastener * 2) / top_force

# 3.3 Bearing Stress (Bolt crushing G10)
width_force = 6 * segment_modules_weight * 40 * G
bearing_area = dia * garolite_thickness
bearing_stress = (width_force / 6) / bearing_area # Distributed across 6 bolt locations
sf_bearing = garolite_strength / bearing_stress

print(f"Bolt Tension SF: {sf_bolt_tension:.4f}")
print(f"Bolt Bearing SF: {sf_bearing:.4f}")

Bolt Tension SF: 26.5526
Bolt Bearing SF: 0.9950


## 4. Euler-Johnson Buckling Analysis

In [46]:
i_walls = 2 * (((ene_segment_width/2 + garolite_thickness/2)**2 * garolite_thickness * (ene_seg_height + (2 * garolite_thickness))))
radius_of_gyration = math.sqrt(i_walls / depth_cross_sec_area)
slenderness_ratio = ene_segment_depth / radius_of_gyration
critical_slenderness = math.pi * math.sqrt((2 * garolite_modulus) / garolite_strength)

if slenderness_ratio < critical_slenderness:
    max_force_buckle = depth_cross_sec_area * (garolite_strength - (garolite_strength**2 * slenderness_ratio**2) / (4 * math.pi**2 * garolite_modulus))
    mode = "Johnson"
else:
    max_force_buckle = (math.pi**2 * garolite_modulus * i_walls) / (ene_segment_depth**2)
    mode = "Euler"

sf_buckling = max_force_buckle / (segment_modules_weight * 40 * G)
print(f"Buckling Mode: {mode}, SF: {sf_buckling:.4f}")

Buckling Mode: Johnson, SF: 57.6237


## 5. Summary Results Table

In [47]:
summary_data = {
    "Load Case": ["Vertical Bending", "Bolt Tension", "Bolt Bearing", "Casing Buckling"],
    "Safety Factor": [sf_lid, sf_bolt_tension, sf_bearing, sf_buckling],
    "Minimum Required": [1.0, 1.0, 1.0, 1.0]
}

df = pd.DataFrame(summary_data)
df["Status"] = df["Safety Factor"].apply(lambda x: "PASS" if x >= 1.0 else "FAIL")
display(df.round(4))

,Load Case,Safety Factor,Minimum Required,Status
0,Vertical Bending,1.7468,1.0,PASS
1,Bolt Tension,26.5526,1.0,PASS
2,Bolt Bearing,0.9950,1.0,FAIL
3,Casing Buckling,57.6237,1.0,PASS
